In [1]:
#import necessary libraries and set pyspark environment

import sys,os,glob,math
import pandas as pd
os.environ["PYSPARK_PYTHON"] = "C:/Users/z001133/AppData/Local/Continuum/anaconda3/python.exe"
os.environ["SPARK_HOME"] = "D:/Public/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")
from pyspark.sql import SparkSession, SQLContext
spark = SparkSession.builder.appName("GrabChallengeSafety").master("local[*]").config('spark.executor.memory','20G').config('spark.driver.memory','10G').getOrCreate()

In [2]:
#load features .csv files in to a dataframe
os.chdir('C:\\Users\\z001133\\IdeaProjects\\pyLearning\\data\\features1')
df_01 = pd.DataFrame()
for file in glob.glob("*.csv"):
    print(file)
    df_00 = pd.read_csv(file)
    df_01 = df_01.append(df_00,ignore_index=True)
    del df_00
#df_02 = spark.createDataFrame(df_01[df_01['bookingID'] == 2])
df_02 = spark.createDataFrame(df_01)
del df_01

part-00000-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00001-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00002-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00003-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00004-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00005-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00006-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00007-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00008-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv
part-00009-e6120af0-10c2-4248-97c4-81baf4304e5c-c000.csv


In [ ]:
#df_02 = spark.createDataFrame(df_01[df_01['bookingID'] == 2])

In [ ]:
#df_02.orderBy("second").show(195)

In [3]:
#calculate magnitude of accelaration for the give accelaration x,y and z axis values
from pyspark.sql.functions import udf
def get_accelaration_magnitude(a,b,c):
    p = (a*a + b*b + c*c)
    return math.sqrt(p)
get_accelaration_magnitude_udf = udf(get_accelaration_magnitude)

In [4]:
#calculate degree of rotation in x,y, and z axis for the give gyrometer x,y and z values
#create a new column speedT for temporary manipulation need down the line
df_02 = df_02.withColumn("degreesX",df_02["gyro_x"]*57.2958) \
.withColumn("degreesY",df_02["gyro_y"]*57.2958) \
.withColumn("degreesZ",df_02["gyro_z"]*57.2958) \
.withColumn("acc_magn",get_accelaration_magnitude_udf(df_02["acceleration_x"],df_02["acceleration_y"],df_02["acceleration_z"])) \
.withColumn("speedT",df_02["Speed"])

In [5]:
#select the columns needed
df_02 = df_02.select("bookingID","second","Accuracy","Bearing","degreesZ","acc_magn","Speed","speedT")

In [6]:
#group data by driving trip id , accuracy of gps , gps bearing position 
#this is to group the behaviour by change in the direction of drive
#calculate average behaviour of degree of rotation, magnitude of accelaration, 
#total time taken in the direction, average speed and distance traveled in the direction

df_02 = df_02.groupby(['bookingID','Accuracy','Bearing']) \
        .agg({
        "degreesZ": "mean",
        "acc_magn": "mean",
        "second": "max",
        "Speed": "mean",
        "speedT": "sum"
        }) \
    .withColumnRenamed("bookingID","bkid") \
    .withColumnRenamed("Bearing","brng") \
    .withColumnRenamed("Accuracy","acrc") \
    .withColumnRenamed("avg(degreesZ)","degreesZ") \
    .withColumnRenamed("avg(acc_magn)","acc_magn") \
    .withColumnRenamed("max(second)","sub_total_time") \
    .withColumnRenamed("avg(Speed)","speed") \
    .withColumnRenamed("sum(speedT)","sub_total_distance") 

In [ ]:
#df_02.orderBy("timehr").show(2)

In [7]:
#convert time , speed and distance in to higher measures [hrs & km's]
df_02 = df_02.withColumn("speedhr",3.6*df_02['speed']) \
    .withColumn("timehr",df_02['sub_total_time']/3600) \
    .withColumn("dist",df_02['sub_total_distance']/1000)
    
df_02 = df_02.withColumn("timehrT",df_02['timehr']) \
    .withColumn("distT",df_02['dist'])

In [8]:
#store data in to a temporary view
df_02.createOrReplaceTempView('df_03')

In [10]:
#function to calculate difference of value for each direction change
from pyspark.sql.functions import udf
def check_first_row(a,b):
    if b != 0:
        return a-b
    else:
        return 0
check_first_row_udf = udf(check_first_row)

In [11]:
# for each change in direction calculate change in 
# bearing , accuracy, magnitude of accelaration, degree of rotation [taken z axis alone hoping it is aligned to the wheel]
# distance , speed

df_04 = spark.sql("""select *,lag(brng,1,0) over (partition by bkid order by timehr) as prev_brng,
     lag(acrc,1,0) over (partition by bkid order by timehr) as prev_acrc,
     lag(acc_magn,1,0) over (partition by bkid order by timehr) as prev_acc_magn,
     lag(degreesZ,1,0) over (partition by bkid order by timehr) as prev_degreesZ,
     lag(dist,1,0) over (partition by bkid order by timehr) as prev_dist,
     lag(speedhr,1,0) over (partition by bkid order by timehr) as prev_speed from df_03""")

In [ ]:
#df_04.orderBy("timehr").show()

In [12]:
#using the above define function calculate the delta values of above features

df_04 = df_04.withColumn('delta_brng',check_first_row_udf(df_04['brng'],df_04['prev_brng']))\
.withColumn('delta_acrc',check_first_row_udf(df_04['acrc'],df_04['prev_acrc']))\
.withColumn('delta_acc_magn',check_first_row_udf(df_04['acc_magn'],df_04['prev_acc_magn']))\
.withColumn('delta_degreesZ',check_first_row_udf(df_04['degreesZ'],df_04['prev_degreesZ']))\
.withColumn('delta_dist',check_first_row_udf(df_04['dist'],df_04['prev_dist']))\
.withColumn('delta_speed',check_first_row_udf(df_04['speedhr'],df_04['prev_speed']))

In [ ]:
#df_04.orderBy("timehr").show()

In [13]:
#grouping data of each direction by drive trip id [booking id] to get the features that has the characteristics of a given driving
#standard deviation in bearing (thats grouped earlier by direction change)
#total number of direction changes
#standard deviation in accuracy of gps
#standard deviation in speed 
#standard deviation in acclearation magnitude
#total time take for the trip
#mean of the time across the direction changes
#average distance across the direction changes
#total distance travelled for the trip
#standard deviation of the changes that happened between the direction changes 
#accuracy,magnitude of accelaration,degree of rotatio,distance & speed

df_05 = df_04.groupBy('bkid') \
        .agg({
        "brng": "std",
        "bkid": "count",
        "acrc": "std",
        "degreesZ": "std",
        "speedhr": "std",
        "acc_magn": "std",
        "timehr": "max",
        "timehrT": "mean",
        "dist": "mean",
        "distT": "sum",
        "delta_brng": "std",
        "delta_acrc": "std",
        "delta_acc_magn": "std",
        "delta_degreesZ": "std",
        "delta_dist": "std",
        "delta_speed": "std"
    }) \
        .withColumnRenamed("bkid","bookID") \
        .withColumnRenamed("stddev(brng)","bearing") \
        .withColumnRenamed("count(bkid)","no_of_turns") \
        .withColumnRenamed("stddev(acrc)","accuracy") \
        .withColumnRenamed("stddev(degreesZ)","degreesZ") \
        .withColumnRenamed("stddev(acc_magn)","acc_magn") \
        .withColumnRenamed("avg(timehrT)","time_per_turn") \
        .withColumnRenamed("max(timehr)","total_time") \
        .withColumnRenamed("stddev(speedhr)","speed") \
        .withColumnRenamed("avg(dist)","distance_per_turn") \
        .withColumnRenamed("sum(distT)","total_ditance") \
        .withColumnRenamed("stddev(delta_brng)","change_in_bearing") \
        .withColumnRenamed("stddev(delta_acrc)","change_in_accuracy") \
        .withColumnRenamed("stddev(delta_acc_magn)","change_in_accelaration") \
        .withColumnRenamed("stddev(delta_degreesZ)","change_in_gyro") \
        .withColumnRenamed("stddev(delta_dist)","change_in_dist") \
        .withColumnRenamed("stddev(delta_speed)","change_in_speed")

In [ ]:
#print(df_05.head())

In [ ]:
#df_05.coalesce(1).write.option("header","true").csv("C:\\Users\\z001133\\IdeaProjects\\pyLearning\\data\\features\\features_transformed.csv")

In [15]:
#read the label values in to a dataframe

os.chdir('C:\\Users\\z001133\\IdeaProjects\\pyLearning\\data\\labels')
df_01 = pd.DataFrame()
for file in glob.glob("*.csv"):
    print(file)
    df_00 = pd.read_csv(file)
    df_01 = df_01.append(df_00,ignore_index=True)
    del df_00
df_02 = spark.createDataFrame(df_01)
del df_01
print(df_02.head())

part-00000-e9445087-aa0a-433b-a7f6-7f4c19d78ad6-c000.csv
Row(bookingID=111669149733, label=0)


In [16]:
#join the features and labels
df_06 = df_05.join(df_02, df_05["bookID"] == df_02["bookingID"])

In [17]:
#store the data at trip level (1 observation per trip) in a .csv file

df_06.coalesce(1).write.option("header","true").csv("C:\\Users\\z001133\\IdeaProjects\\pyLearning\\data\\labels\\consolidated.csv")